In [1]:
import sys
import numpy as np
from pydub import AudioSegment
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
from librosa import display
import glob

C:\Users\lashen\AppData\Local\Continuum\anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Convert Spectrogram to Audio

#### Perform ISTFT on spectrogram 

In [47]:
def convert_spect_to_time_series(spectrogram_list):
    
    
    time_series_list = []
    
    for spectrogram in spectrogram_list:
        reshaped_spectrogram = np.reshape(spectrogram, (1025, 22))
        y_hat = librosa.istft(reshaped_spectrogram)
        time_series_list.append(y_hat)
    
    return time_series_list

#### Write time series to wav file

In [64]:
def write_time_series_to_wav(time_series, sampling_rate):
    file_prefix = 'generated_'
    file_names_list = []
    
    i = 0
    for series in time_series:
        file_name = file_prefix + str(i) + '.wav'
        librosa.output.write_wav(file_name, series, sampling_rate)
        file_names_list.append(file_name)
        i = i + 1
        
    return file_names_list

#### Stitch the wav files into one time series

In [65]:
def stitch_wavs_together(file_names):
    
    wav_clips = []
    clip_file_list = glob.glob('generated_*.wav')
    print (clip_file_list)
    
    for file in clip_file_list:
        print (file)
        clip = AudioSegment.from_wav(file)
        wav_clips.append(clip)
        
    whole_clip = wav_clips[0]
    
    for i in range(1, len(wav_clips)):
        whole_clip = whole_clip + wav_clips[i]
        
    whole_clip.export('whole_clip.wav', format='wav')
    

#### Main function to convert spectrograms to wav

In [66]:
def convert_spect_to_wav(file_name):
    spectrogram_array = np.load(file_name)
    print (spectrogram_array.shape)
    spectrogram_list = []
    
    for spectrogram in spectrogram_array:
        #print (spectrogram.shape)
        spectrogram_list.append(spectrogram)
    
    #spectrogram_list = spectrogram_array.tolist()
    #print (spectrogram_list)
    
    # Convert spectrogram to time series
    time_series = convert_spect_to_time_series(spectrogram_list)
        
    # Write time series out to wav files
    file_names = write_time_series_to_wav(time_series, 22050)
    print (file_names)
    
    # Piece together the wav files
    stitch_wavs_together(file_names)

In [67]:
convert_spect_to_wav('../dataset_processed/bdl_spectrogram_array.npy')

(6, 205, 110)
['generated_0.wav', 'generated_1.wav', 'generated_2.wav', 'generated_3.wav', 'generated_4.wav', 'generated_5.wav']
['generated_0.wav', 'generated_1.wav', 'generated_2.wav', 'generated_3.wav', 'generated_4.wav', 'generated_5.wav']
generated_0.wav


FileNotFoundError: [WinError 2] The system cannot find the file specified